In [1]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

from Libs.libs import *

In [2]:
## Lamborghini Tool
from Schema import *
def Lamborghini(query: str):
    template = """
    You are a tool that answer queries related to Lamborghini vehicle. 
    If the query is other than Lamborghini, say i do not know.
    You are given a human query: {query}

        provide response in 3 sentences in under 30 words and  concise, precise.


                    """
    prompt_temp = ChatPromptTemplate.from_template(template=template
                                                   )

    chain = RunnableMap({
        "query": lambda x: x['query'],

    }) | prompt_temp | llm | string_parser
    

    response = chain.invoke({"query": query})
    print(response)
    return response



vehicle_lambo_Tool = StructuredTool.from_function(
        name='vehicleLamboTool',
        func=Lamborghini,
        description="A tool that responds to query related to only Lamborghini.",
        args_schema=QueryInput,
        return_direct=True
    )






def mercedes(query: str):
    template = """
    You are a tool that answer queries related to specs of mercedes vehicle. 
    If the query is other than mercedes, say i do not knoe
    You are given a human query: {query}
         provide response in 3 sentences in under 30 words and  concise, precise.
                    """
    prompt_temp = ChatPromptTemplate.from_template(template=template
                                                   )

    chain = RunnableMap({
        "query": lambda x: x['query'],

    }) | prompt_temp | llm |string_parser

    response = chain.invoke({"query": query})
    print(response)
    return response



vehicleTool = StructuredTool.from_function(
        name='vehicleTool',
        func=mercedes,
        description="A tool that responds to query related to only Mercedes.",
        args_schema=QueryInput,
        return_direct=True
    )

In [3]:
tools_available = [vehicle_lambo_Tool, vehicleTool]

## Agent Node

In [4]:
from langchain_core.messages import HumanMessage


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
    }

## Supervisor agent

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

members = ["Mercedes", "Lamborghini"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members


class routeResponse(BaseModel):
    next: Literal["Mercedes", "Lamborghini", "FINISH"]


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))


# llm = ChatOpenAI(model="gpt-4o")


def supervisor_agent(state):
    supervisor_chain = prompt | llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

In [6]:
import functools
import operator
from typing import Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


mercedes = create_react_agent(llm, tools=[vehicleTool])
lamborghini = create_react_agent(llm, tools=[vehicle_lambo_Tool])
lamborghini_node = functools.partial(agent_node, agent=mercedes, name="Mercedes")
lamborghini_node = functools.partial(agent_node, agent=lamborghini, name="Lamborghini")

# # NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
# code_agent = create_react_agent(llm, tools=[python_repl_tool])
# code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Lamborghini", lamborghini_node)
workflow.add_node("Mercedes", mercedes)
workflow.add_node("supervisor", supervisor_agent)

In [7]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

In [8]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Tell me about mercedes")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Mercedes'}}
----
Mercedes-Benz is a luxury automotive brand known for its high-quality vehicles, innovative technology, and performance. The brand offers a wide range of cars, SUVs, and electric vehicles. It combines elegance with cutting-edge engineering.
{'Mercedes': {'messages': [HumanMessage(content='Tell me about mercedes', additional_kwargs={}, response_metadata={}, id='7e2d37b9-ffd6-4e86-b6ab-48f2fcebb4f4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jXwkqn3jl90F055AMEpnG8To', 'function': {'arguments': '{"query":"Tell me about Mercedes."}', 'name': 'vehicleTool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 66, 'total_tokens': 84, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'tool_calls', 'logprobs': None},

In [ ]:



from Libs.libs import *


from Tools.availability_by_doctor import *
from Tools.availability_by_specialization import *
from Tools.booking import book_appointment

class MessagesState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]
    senderId:str
    history:str
    
    
tools_available = [book_appointment,check_availability_by_doctor, check_availability_by_specialization]
# tool = [mercedes_tool]
tool_node = ToolNode(tools=tools_available)
model = llm.bind_tools(tools = tools_available, strict=True)

def read_human_feedback(state):
    # if state['messages'][-1].tool_calls == []:
    #     logger.info("AI: "+ state['messages'][-1].content)
    #     user_msg = input("Reply: ")
    
    
    # history = {"history":[]}
    # history = getData("abcsdd")
    # if history is None:
    #     history = {'history':[]}
    # history_new = {"human_feedback":f"{state['messages'][0].content}", "ai":f"{state['messages'][-1].content}"}



    # history = history_new.append(history['history'])
    # history['history'].append(history_new)
    # print(history)

        
        
    
    # history['history'].append(history_new)
    
    # setData(state['senderId'], history)
    
    # if len(history['history'])>4:
    #     length = len(history['history']) - 4
    #     for i in range(length):
    #         history["history"].pop(0)
    
    # a = getData("abcsdd")
    # combined_history.append(a)
    # {"history":combined_history}
        
    
    
    # print()
    # print(state)
    # print("history",history)
    # print()
    return state
    #     pass

def call_model(state: MessagesState):
    
    
    
    print("From call_model the state is:",state['senderId'])
    s = getData(state["senderId"])
    state["history"] = s
    

    # history = {"human_feedback":state["messages"][1], "AI":"This is ai response"}
    messages = [SystemMessage(content=f"You are helpful assistant.\n.As reference, today is {datetime.now().strftime('%Y-%m-%d %H:%M, %A')}\nKeep a friendly, professional tone.\nAvoid verbosity.\nConsiderations:\n- Don´t assume parameters in call functions that it didnt say.\n- MUST NOT force users how to write. Let them write in the way they want.\n- The conversation should be very natural like a secretary talking with a client.\n- Call only ONE tool at a time.")] + state['messages']
    # messages = [SystemMessage(content="You are a helpful assistant. As reference, today is {datetime.now().strftime('%Y-%m-%d %H:%M, %A')}. Always use tools to answer the queries")]

    response = model.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "human_feedback"]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return "human_feedback"




def should_continue_with_feedback(state: MessagesState) -> Literal["agent", "end", "human_feedback"]:
    messages = state['messages']
    last_message = messages[-1]
    if isinstance(last_message, dict):
        if last_message.get("type","") == 'human_feedback':
            return "agent"
    if (isinstance(last_message, HumanMessage)):
        return "agent"
    if (isinstance(last_message, AIMessage)):
        return "end"
    return "end"



def graph(query:str, senderId:str):
    workflow = StateGraph(MessagesState)
    workflow.add_node("agent",call_model)
    workflow.add_node("tools",tool_node)
    workflow.add_node("human_feedback", read_human_feedback)


    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {"human_feedback":"human_feedback",
        "tools":"tools"}
    )
    workflow.add_conditional_edges(
        "human_feedback",
        should_continue_with_feedback,
        {"agent":"agent","end":END}
    )

    workflow.add_edge("tools","agent" )


    workflow.set_entry_point('agent')



    graph = workflow.compile()
    inputs = {"messages":[HumanMessage(content=query)], "senderId":senderId}

    for response in graph.stream(inputs):
        try:
            if "__end__" not in response:
                print(response)
                # if 'human_feedback' in response:
                token_usage =response['human_feedback']['messages'][-1].response_metadata['token_usage']
                final_response =  response['human_feedback']['messages'][-1].content
                    
                print("-----")
                # history = {"human_feedback":query, "ai":final_response}
                return {"result": final_response, "token_usage":token_usage}
        except Exception as e:
            print("error", e)



